# Image and Video Captioning Model Structures

### image encoder is a CNN and the decoder is an RNN
![](./images/snap_lecture_structure1.png)
### Trasnfer learning applied to the CNN model, we only train the final layer
![](./images/snap_lecture_structure2.png)
### Same technology applied to video streams, several frames from a video are passed through CNN, the Feature map from the frames are averaged then to create a single output representation of the frames. The rest is same as image captioning
![](./images/snap_lecture_structure3.png)